In [1]:
with open('input.txt', "r") as f:
    lines = f.read().rstrip().split('\n')
    
lines[:10]

In [2]:
from collections import OrderedDict

def parse_mask(line):
    return line.split('= ')[1]


def parse_mem(line):
    mem_str, val = line.split(' = ')
    val = int(val)

    import re
    mem = int(re.findall(r'\d+', mem_str)[0])
    return mem, val

G = OrderedDict()


for line in lines:
    if line.startswith('mask'):
        mask = parse_mask(line)
        G[mask] = []
    else:
        mem, val = parse_mem(line)
        G[mask].append((mem, val))
list(G.keys())[:5]

In [3]:
def get_changed_bits(mask):
    changed_bits = {}
    for i in range(len(mask)):
        if mask[i] != 'X':
            changed_bits[i] = mask[i]
    return changed_bits

get_changed_bits(list(G.keys())[0])        

In [4]:
def dec_to_bin(x):
    return str(bin(x)[2:].zfill(36))

def bin_to_dec(bin_str):
    return int(bin_str, 2)

print(dec_to_bin(8))
print(bin_to_dec('000000000000000000000000000000001000'))


000000000000000000000000000000001000
8


In [5]:
def apply_mask(bin_str, changed_mask_dict):
#     print(bin_str)
    bin_str = list(bin_str)
    for k, v in changed_mask_dict.items():
#         print(k, v)
        bin_str[k] = v
    return "".join(bin_str)

apply_mask(dec_to_bin(101), get_changed_bits(list(G.keys())[0]))

'000100011010101100101001010011110000'

In [6]:
def update_memory(mask_dict, mem_addr, suggested_val, mask_func):
    final_mem_value_bin = mask_func(dec_to_bin(suggested_val), mask_dict)
    MEM[mem_addr] = bin_to_dec(final_mem_value_bin)
    

In [10]:
MEM = {}


for mask in G.keys():
    mask_dict = get_changed_bits(mask)
    for addr, val in G[mask]:
        update_memory(mask_dict, addr, val, apply_mask)
sum(MEM.values())

9879607673316

## Part 2

In [17]:
def get_combinations(poss_vals_set, k, buffer):
    get_combinations_recursive(poss_vals_set, "", k, buffer)
    return buffer


def get_combinations_recursive(poss_vals_set, s, k, buffer):
    if len(s) == k:
        buffer.add(s)
        return

    for val in poss_vals_set:
        get_combinations_recursive(poss_vals_set, s + val, k, buffer)
    return



In [18]:
def get_modified_addresses(bin_str):
    result = []
    combinations = set()
    floating_idxs = []
    for i in range(len(bin_str)):
        if bin_str[i] == 'X':
            floating_idxs.append(i)
            
    combs = get_combinations(['0', '1'], len(floating_idxs), set())
    
    for comb in combs:
        new_string = bin_str.copy()
        for i in range(len(comb)):
            new_string[floating_idxs[i]] = comb[i]
        result.append("".join(new_string))
    return list(map(bin_to_dec, result))
        

def apply_mask_v2(bin_str, mask):
    result = list(bin_str)
    floating_idxs = []
    for i in range(len(mask)):
        if mask[i] == '0':
            continue
        elif mask[i] == '1':
            result[i] = '1'
            
        elif mask[i] == 'X':
            result[i] = 'X'
    addresses = get_modified_addresses(result)       
    return addresses

val = '000000000000000000000000000000011010'    
mask = '00000000000000000000000000000000X0XX'
k = apply_mask_v2(val, mask)
sorted(k)

[16, 17, 18, 19, 24, 25, 26, 27]

In [19]:
def update_mem_v2(mask, base_addr, val):
    new_addresses = apply_mask_v2(base_addr, mask)
    for add in new_addresses:
        MEM[add] = bin_to_dec(val)

In [20]:
MEM = {}

for mask in G.keys():
    for addr, val in G[mask]:
        update_mem_v2(mask, dec_to_bin(addr), dec_to_bin(val))
sum(MEM.values())

3435342392262